# Evaluate Predictions

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import os

## Load custom modules

In [2]:
from nnssa.constants import *
from nnssa.evaluate import post_processing

## Load other modules

In [3]:
import peakutils
import mir_eval

## Constants

In [4]:
ROOT = os.path.join(os.getcwd(), '..', '..') 

## Progress bars!

In [5]:
from tqdm import tqdm
tqdm.pandas()

/Users/admin/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Load dataset

In [6]:
harmonix_beats = pd.read_pickle(os.path.join(ROOT, SUB_DIVS_DIR, 'bars', 'harmonix.p'))
harmonix_beats = harmonix_beats.head(50)
harmonix_beats.head()

,File,Sub_Divisions,Binary_Labels,Weighted_Labels,Weights,IDS,Beat_times,Labels,BPM
0,0001_12step,bars/harmonix/0001_12step.npy,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 0.5, 3.0, 0.5, 1.0, 1.0, 1.0, ...","[0001_12step, 0001_12step, 0001_12step, 0001_1...","[0.0, 0.5309729999999999, 1.0619459999999998, ...","[0.0, 8.495567999999999, 25.486704, 42.4753280...",113
1,0003_6foot7foot,bars/harmonix/0003_6foot7foot.npy,"[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0003_6foot7foot, 0003_6foot7foot, 0003_6foot7...","[2.857108, 3.571394, 4.28568, 4.99996600000000...","[2.857108, 8.571396, 31.428548, 37.14283599999...",84
2,0004_abc,bars/harmonix/0004_abc.npy,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0004_abc, 0004_abc, 0004_abc, 0004_abc, 0004_...","[2.666656, 3.238084, 3.952369, 4.597529, 5.242...","[2.666656, 28.300542999999998, 58.263180000000...",94
3,0006_aint2proud2beg,bars/harmonix/0006_aint2proud2beg.npy,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0006_aint2proud2beg, 0006_aint2proud2beg, 000...","[0.0, 0.572203, 1.144406, 1.716609, 2.288812, ...","[0.0, 27.4652, 45.203726, 63.518522999999995, ...",105
4,0008_america,bars/harmonix/0008_america.npy,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 0.5, 3.0, 0.5, 1.0, 1.0, 1.0, ...","[0008_america, 0008_america, 0008_america, 000...","[3.871208, 4.359011, 4.846814, 5.338616, 5.830...","[3.871208, 10.56504, 33.217138, 56.85190400000...",136


## Choose a single track

In [7]:
billionaire = harmonix_beats.iloc[17]
filename = billionaire.Sub_Divisions
billionaire.Sub_Divisions = np.load(os.path.join(ROOT, SUB_DIVS_DIR, filename), allow_pickle=True)

/Users/admin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## Build model

In [8]:
import tensorflow as tf
ROOT = os.path.join(os.getcwd(), '..', '..') 
model = tf.keras.models.load_model(os.path.join(ROOT, 'models', 'full_model.h5'))

## Get Predictions

In [9]:
billionaire_preds = model.predict(billionaire.Sub_Divisions)

## Filter predictions

## Get beat times & segments

In [10]:
beat_times = billionaire.Beat_times
segments = billionaire.Labels

## Get peaks

In [11]:
p = post_processing(billionaire_preds.squeeze())
peaks = peakutils.indexes(p, min_dist=4, thres=0.05)

## Split peaks to bars

In [12]:
peaks = [(p * 4) for p in peaks if (p * 4) < len(beat_times)]

## Get onsets

In [13]:
boundaries = beat_times[peaks]

## Evaluate

In [14]:
f_score, precision, recall = mir_eval.onset.f_measure(segments, boundaries, window=3)

In [15]:
print("F-score: ", f_score)
print("Precision: ", precision)
print("Recall: ", recall)

F-score:  0.875
Precision:  0.875
Recall:  0.875


## Print boundary predictions

In [16]:
boundaries

array([  8.197973  ,  30.25747595,  52.31636445,  85.42290845,
       118.50583245, 154.38840845, 173.70702329, 195.76560729])